Doc chain to generate answer from llm based on provided context and query asked
Retrieval chain for getting answer by comparing closeness of vector from the stored data (KNN concept)
Ret+Doc chain concept see below for all 3 concepts

In [1]:
import os

import bs4
from dotenv import load_dotenv
load_dotenv()
os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")
#os.environ["GOOGLE_API_KEY"]=os.getenv("GEMINI_KEY")
#Langchain Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB

In [2]:
#data ingestion
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader=WebBaseLoader("https://docs.smith.langchain.com/administration/tutorials/manage_spend")
loader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='\n\n\n\n\nOptimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended R

In [4]:
#data transformation
from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
splitted=splitter.split_documents(docs)
splitted

[Document(metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\n\n\nSkip to main contentOur Building Ambient Agents with LangGraph course is now available on LangChain Academy!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationTutorialsOptimize tracing spend on LangSmithHow-to GuidesSetupConceptual GuideSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceAdministrationTutorialsOptimize tracing spend on LangSmithOn this pageOptimize tracing spend on LangSmith\nRecommended ReadingBefo

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000002669FF4A660>, model='models/embedding-001', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [6]:
from langchain_community.vectorstores import FAISS
db=FAISS.from_documents(splitted,embeddings)
db

In [7]:
#asking for a answer to query
query="LangSmith has two usage limits: total traces and extended"
result=db.similarity_search(query)
result[1].page_content

'As we can see, the maximum cost is cut from ~40k per month to ~7.5k per month, because we no longer allow as many expensive\ndata retention upgrades. This lets us be confident that new users on the platform will not accidentally cause cost to balloon.\nnoteThe extended data retention limit can cause features other than traces to stop working once reached. If you plan to\nuse this feature, please read more about its functionality here.\nSet dev/staging limits and view total spent limit across workspaces\u200b\nFollowing the same logic for our dev and staging environments, we set limits at 10% of the production\nlimit on usage for each workspace.\nWhile this works with our usage pattern, setting good dev and staging limits may vary depending on\nyour use case with LangSmith. For example, if you run evals as part of CI/CD in dev or staging, you may\nwant to be more liberal with your usage limits to avoid test failures.'

In [8]:
from langchain_google_genai import GoogleGenerativeAI
llm=GoogleGenerativeAI(model="gemini-1.5-flash")
llm

GoogleGenerativeAI(model='gemini-1.5-flash', google_api_key=SecretStr('**********'), client=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000266AF4B0190>, default_metadata=(), model_kwargs={}))

now we saw that in prompt we can design our llm to act as a specific " engg or a thing" while answering the queries
now to get output from a specific content we can use document chain

In [9]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| GoogleGenerativeAI(model='gemini-1.5-flash', google_api_key=SecretStr('**********'), client=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000266AF4B0190>, default_metadata=(), model_kwargs={}))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[]

In [10]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"LangSmith has two usage limits: total traces and extended",
    "context":[Document(page_content="LangSmith has two usage limits: total traces and extended traces. These correspond to the two metrics we've been tracking on our usage graph. and it can help us to track our billings also")]
})

'LangSmith has two usage limits: total traces and extended traces, which are reflected in their usage graph and used for billing tracking.'

now we gave our own context and asked a query based on it the llm answered this is our document_chain
and
earlier we used retreiver to get answer of the query from the stored dat
 Q : What if we want a llm to generate an answer for the query from the stored data
     this needs to used retriever to get relevant data using vector close distance
     then document chain to get all that data and generate an answer that relates to query

we asked query--->goes to retriever ->>> search in vectore store first--> get all the relevant data ->feed the doc chain with that output data  and question  to get the output

In [11]:
db

In [12]:
retriever=db.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000266AE3C6BA0>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
  

In [14]:
#getting the response from the llm
response=retrieval_chain.invoke({"input":"LangSmith has two usage limits: total traces and extended"})
response['answer']

'Based on the provided text, LangSmith offers two usage limits:  total traces and extended retention traces.  These limits can be set per workspace in the Settings -> Usage and Billing -> Usage configuration section.  The "total traces" limit controls the overall number of traces sent to LangSmith, while the "extended retention traces" limit affects traces with extended 400-day data retention.  Setting these limits helps control costs, as seen by an example where maximum monthly cost was reduced from ~$40k to ~$7.5k by limiting expensive data retention upgrades.  The text also suggests setting lower limits for development and staging environments (e.g., 10% of the production limit), though this may need adjustment depending on usage patterns.  The system tracks usage and costs per workspace (referred to as "tenant_id"), allowing for granular cost monitoring and control.'

In [15]:
response['context']

[Document(id='5e48d572-481c-49d8-8115-72f3fc0c0e35', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimization 2: limit usage\u200b\nIn the previous section, we managed data retention settings to optimize existing spend. In this section, we will\nuse usage limits to prevent future overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over spend.\nTo set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the\nbottom of the page that lets you set usage limits per workspace. For each workspace, the two limits appear, along\nwith a cost estim

In [16]:
#provides context +answer both
response

{'input': 'LangSmith has two usage limits: total traces and extended',
 'context': [Document(id='5e48d572-481c-49d8-8115-72f3fc0c0e35', metadata={'source': 'https://docs.smith.langchain.com/administration/tutorials/manage_spend', 'title': 'Optimize tracing spend on LangSmith | 🦜️🛠️ LangSmith', 'description': 'Before diving into this content, it might be helpful to read the following:', 'language': 'en'}, page_content='Optimization 2: limit usage\u200b\nIn the previous section, we managed data retention settings to optimize existing spend. In this section, we will\nuse usage limits to prevent future overspend.\nLangSmith has two usage limits: total traces and extended retention traces. These correspond to the two metrics we\'ve\nbeen tracking on our usage graph. We can use these in tandem to have granular control over spend.\nTo set limits, we navigate back to Settings -> Usage and Billing -> Usage configuration. There is a table at the\nbottom of the page that lets you set usage limits